In [5]:
import os
# to use or not to use GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
!sudo pip install tensorflow --upgrade
# packages for model graph visualization
#!pip install -q pydot
# install graphviz https://graphviz.gitlab.io/download/ 

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
!pip install -q -U tensorflow-text
import tensorflow_text as text
!pip install -q tf-models-official

import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt

#save_path = "/content/drive/My Drive/Colab Notebooks/"

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
save_path = "/content/drive/My Drive/Colab Notebooks/"

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [9]:
# checking if we have access to a GPU

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
#@title Data Loading & Preprocessing


In [34]:
with open(save_path+'project_data_regex.json') as f:
    data = json.load(f)
    # shuffle data
    random.shuffle(data)
    # create description and assignee list
    desc_data = []
    assignee_data = []
    for item in data:
        desc_data.append(item['description'])
        assignee_data.append(item['assignee'])

In [35]:
print('sample assignees:', assignee_data[0:2])
print('sample descriptions:', desc_data[0:2])

sample assignees: ['Harsha', 'Dongjin Lee']
sample descriptions: ['We always see the following error in state change log on shutting down the last broker 2015 01 20 13 21 04 036 ERROR Controller 0 epoch 3 initiated state change for partition test 0 from OfflinePartition to OnlinePartition failed state change logger kafka common NoReplicaOnlineException No replica for partition test 0 is alive Live brokers are Set Assigned replicas are List 0 at kafka controller OfflinePartitionLeaderSelector selectLeader PartitionLeaderSelector scala 75 at kafka controller PartitionStateMachine electLeaderForPartition PartitionStateMachine scala 357 at kafka controller PartitionStateMachine kafka controller PartitionStateMachine handleStateChange PartitionStateMachine scala 206 at kafka controller PartitionStateMachine anonfun triggerOnlinePartitionStateChange 3 apply PartitionStateMachine scala 120 at kafka controller PartitionStateMachine anonfun triggerOnlinePartitionStateChange 3 apply PartitionSta

In [36]:
# transform description data to tensorflow dataset
descriptions = tf.data.Dataset.from_tensor_slices(desc_data)
for input in descriptions.take(2):
    print(input)

tf.Tensor(b'We always see the following error in state change log on shutting down the last broker 2015 01 20 13 21 04 036 ERROR Controller 0 epoch 3 initiated state change for partition test 0 from OfflinePartition to OnlinePartition failed state change logger kafka common NoReplicaOnlineException No replica for partition test 0 is alive Live brokers are Set Assigned replicas are List 0 at kafka controller OfflinePartitionLeaderSelector selectLeader PartitionLeaderSelector scala 75 at kafka controller PartitionStateMachine electLeaderForPartition PartitionStateMachine scala 357 at kafka controller PartitionStateMachine kafka controller PartitionStateMachine handleStateChange PartitionStateMachine scala 206 at kafka controller PartitionStateMachine anonfun triggerOnlinePartitionStateChange 3 apply PartitionStateMachine scala 120 at kafka controller PartitionStateMachine anonfun triggerOnlinePartitionStateChange 3 apply PartitionStateMachine scala 117 at scala collection TraversableLike

In [37]:
# target vectorize assignee data
assignee_dict = {assignee: i for i, assignee in enumerate(list(set(assignee_data)))}

In [38]:
# create target vector
assignee_vector = [assignee_dict[assignee] for assignee in assignee_data]
assignee_vector[:10]

[4, 23, 2, 33, 30, 4, 19, 24, 21, 33]

In [39]:
# assignee target vector to one hot vector
assignee_tensor = tf.data.Dataset.from_tensor_slices(assignee_vector)

for assignee in assignee_tensor.take(2):
    print(assignee)

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(23, shape=(), dtype=int32)


In [40]:
#@title BOW


In [41]:
input_vectorizer = layers.TextVectorization(
    split = "whitespace",
    output_mode='multi_hot')

In [42]:
input_vectorizer.adapt(descriptions)

In [43]:
output_vectorizer = layers.CategoryEncoding(num_tokens = len(assignee_dict), output_mode="one_hot")

In [44]:
for description in descriptions.take(2):
    print(description)
    print(input_vectorizer(description))

tf.Tensor(b'We always see the following error in state change log on shutting down the last broker 2015 01 20 13 21 04 036 ERROR Controller 0 epoch 3 initiated state change for partition test 0 from OfflinePartition to OnlinePartition failed state change logger kafka common NoReplicaOnlineException No replica for partition test 0 is alive Live brokers are Set Assigned replicas are List 0 at kafka controller OfflinePartitionLeaderSelector selectLeader PartitionLeaderSelector scala 75 at kafka controller PartitionStateMachine electLeaderForPartition PartitionStateMachine scala 357 at kafka controller PartitionStateMachine kafka controller PartitionStateMachine handleStateChange PartitionStateMachine scala 206 at kafka controller PartitionStateMachine anonfun triggerOnlinePartitionStateChange 3 apply PartitionStateMachine scala 120 at kafka controller PartitionStateMachine anonfun triggerOnlinePartitionStateChange 3 apply PartitionStateMachine scala 117 at scala collection TraversableLike

In [45]:
for bow_assignee in assignee_tensor.take(2):
    print(bow_assignee)
    print(output_vectorizer(bow_assignee))

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(23, shape=(), dtype=int32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)


In [46]:
bow_dataset = tf.data.Dataset.zip((descriptions, assignee_tensor))

def vectorize_dataset(description, assignee):

  return input_vectorizer(description), output_vectorizer(assignee)

bow_dataset = bow_dataset.map(vectorize_dataset)

for description, assignee in bow_dataset.take(2):
    print(description, assignee)

tf.Tensor([0. 1. 1. ... 0. 0. 0.], shape=(23690,), dtype=float32) tf.Tensor(
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor([0. 1. 1. ... 0. 0. 0.], shape=(23690,), dtype=float32) tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)


In [47]:
bow_dataset = bow_dataset.shuffle(len(list(bow_dataset)), seed = 42)

In [48]:
# batching the dataset
BATCH_SIZE = 32
VAL_SIZE = 0.15
bow_dataset = bow_dataset.batch(batch_size=BATCH_SIZE)

for line, label in bow_dataset.take(2):
    print(line.shape)
    
print(len(list(bow_dataset)))

(32, 23690)
(32, 23690)
136


In [49]:
# splitting the dataset into validation and test data
val_size = round(VAL_SIZE*(len(list(bow_dataset))))

val_dataset = bow_dataset.take(val_size) 
train_dataset = bow_dataset.skip(val_size)

In [50]:
len(list(train_dataset))

116

In [51]:
vocab_size = input_vectorizer.vocabulary_size()
bow_model = tf.keras.Sequential([
  tf.keras.layers.Dense(1024, input_shape=(vocab_size,), activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(len(assignee_dict), activation = "softmax")
])

In [52]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [53]:
bow_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [54]:
bow_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 1024)              24259584  
                                                                 
 dense_3 (Dense)             (None, 256)               262400    
                                                                 
 dense_4 (Dense)             (None, 64)                16448     
                                                                 
 dense_5 (Dense)             (None, 36)                2340      
                                                                 
Total params: 24,540,772
Trainable params: 24,540,772
Non-trainable params: 0
_________________________________________________________________


In [55]:
for inputs,targets in train_dataset.take(1):
    print(inputs)
    outputs = bow_model(inputs)

    print(outputs.shape)
    print(outputs)
    print(targets)

tf.Tensor(
[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]], shape=(32, 23690), dtype=float32)
(32, 36)
tf.Tensor(
[[0.02861565 0.02629971 0.02743287 ... 0.02786437 0.02586665 0.02886937]
 [0.02937284 0.02639755 0.02856585 ... 0.02714753 0.02675392 0.02639784]
 [0.02828984 0.02755247 0.02850856 ... 0.02628703 0.02464152 0.02698893]
 ...
 [0.02837916 0.0269478  0.02784633 ... 0.02737439 0.02685091 0.02732493]
 [0.02898069 0.0269969  0.02804549 ... 0.0279605  0.02682493 0.02719238]
 [0.02852238 0.02567954 0.02968998 ... 0.02772972 0.02389753 0.02663116]], shape=(32, 36), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(32, 36), dtype=float32)


In [56]:
bow_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  api_dispatcher = getattr(op_dispatch_handler, TYPE_BASED_DISPATCH_ATTR,


113/116 [============================>.] - ETA: 0s - loss: 3.2390 - accuracy: 0.1579WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f52c012b200> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
116/116 [==============================] - 7s 38ms/step - loss: 3.2306 - accuracy